In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/attention-model-pl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/attention-model-pl

/content/drive/MyDrive/attention-model-pl


In [3]:
!pip install -r requirements.txt

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
%ls data

data_module.py  dataset.py  __pycache__/


In [6]:
from config import get_config

cfg = get_config()
cfg['batch_size'] = 16
cfg['preload'] = None
cfg['num_epochs'] = 30

from data.data_module import *
from models.lightning_model import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import torch

In [7]:
datamodule = BilingualDataModule(cfg)
datamodule.setup()

Max length of source language: 309
Max length of target language: 274


In [8]:
tk_src = datamodule.tokenizer_src  # Assuming datamodule is your instance of BilingualDataModule
tk_tgt = datamodule.tokenizer_tgt

src_vocab_size = tk_src.get_vocab_size()
tgt_vocab_size = tk_tgt.get_vocab_size()

model = LitModel(src_vocab_size, tgt_vocab_size)

In [9]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [10]:
# Tensorboard logger
tb_logger = TensorBoardLogger(save_dir="logs/", name="model")
# Initialize the Lightning Trainer

trainer = pl.Trainer(max_epochs=cfg["num_epochs"],accelerator="auto",
        devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
        logger=tb_logger,
        callbacks=[
            ModelCheckpoint(dirpath=cfg['model_folder'], verbose=True),
            LearningRateMonitor(logging_interval="step", log_momentum=True), TQDMProgressBar(refresh_rate=10)])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule)

Max length of source language: 309
Max length of target language: 274


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params
--------------------------------------
0 | model | Transformer | 75.1 M
--------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


--------------------------------------------------------------------------------
SOURCE: The men went to the side-table in the dining-room, on which stood bottles with six kinds of vodka and plates with as many sorts of cheese with and without silver cheese-knives, caviar, herrings, different kinds of tinned delicacies, and slices of French rolls.
TARGET: Gli uomini andarono in sala da pranzo e si accostarono alla tavola con gli antipasti, coperta di sei qualità di vodka e di altrettante specie di formaggi con le palettine d’argento o senza, i caviali, le aringhe, le conserve di varie specie e i piatti con le fettine di pane francese.
PREDICTED: potevate potevate trovarci trovarci trovarci trovarci trovarci sono sono trovarci trovarci trovarci trovarci trovarci trovarci trovarci trovarci trovarci trovarci trovarci sono sono sono trovarci trovarci trovarci trovarci descritta descritta esaltazione sono sono sono sono trovarci descritta trovarci descritta trovarci sono sono sono trovarci 

...

--------------------------------------------------------------------------------
SOURCE: This is why!' suddenly shouted Nicholas at the mention of the name. 'This is why.
TARGET: Ah, ecco perché! — gridò ad un tratto Nikolaj, sentendo pronunciare il nome di Sergej Ivanyc — ecco a che scopo...
PREDICTED: — Ma non è vero ! — disse Stepan Arkad ’ ic , — non è più di nuovo .
--------------------------------------------------------------------------------
SOURCE: When once more alone, I reviewed the information I had got; looked into my heart, examined its thoughts and feelings, and endeavoured to bring back with a strict hand such as had been straying through imagination's boundless and trackless waste, into the safe fold of common sense.
TARGET: Appena fui sola mi diedi a ripassare nella mente ciò che avevami detto la signora Fairfax. Guardai nel mio cuore, esaminai pensieri e sentimenti e con mano ferma cercai di ricondurre sulla via del buon senso quelli che l'immaginazione aveva lascia

Validation: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
SOURCE: It was the first of June; yet the morning was overcast and chilly: rain beat fast on my casement.
TARGET: Benché si fosse al 1° di giugno, la mattinata era fredda e nuvolosa e la pioggia batteva contro i vetri.
PREDICTED: Era la mattina in casa , ma il giorno era un letto e il mio letto .
--------------------------------------------------------------------------------
SOURCE: Bar. falling."
TARGET: Barometro in discesa».
PREDICTED: .
--------------------------------------------------------------------------------
SOURCE: "Cease that chatter, blockhead! and do my bidding."
TARGET: — Cessate il vostro chiacchierìo stupido e ubbidite.
PREDICTED: — È un po ' e mi .
--------------------------------------------------------------------------------
SOURCE: Are you fond of it?"
TARGET: Vi diletta quella lettura?
PREDICTED: Avete detto che cosa ?
--------------------------------------------------------------

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 581, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 990, in _run
    results = self._run_stage()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1036, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py", line 202, in run
    self.advance()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py", line 359, in advance
    self.epoch_loop.run(self._data_fetcher)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/training_epoch_loop.py", line 136